<a href="https://colab.research.google.com/github/T-King-00/Gp-AutomationOfBaTasks/blob/tony/extractActorsNounsVerbs%5Bv2%5D16-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:
! pip3 install PyGithub
from github import Github

import requests
response = requests.get('https://raw.githubusercontent.com/T-King-00/Gp-AutomationOfBaTasks/tony/university.txt')
file = response.text


! pip install -U spacy
from spacy.language import Language
import spacy
from spacy.matcher import Matcher
from spacy.symbols import nsubj, VERB



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [127]:
import spacy

pronouns = {'her ':'she ', 'him ':'he ', 'whom ':'who ', 'me ': 'I ', 'us ':'we ', 'them ':'they '}

nlp = spacy.load('en_core_web_sm')


doc = nlp(file)

for sent in doc.sents:
    try:
        sent_list = []
        for token in sent:
            sent_list.append(token)

        main_verb_index = None
        agent_index = None
        recipient_index = None
        agent = ''
        append = False
        inflection = ''
        agent_article = ''
        start_word = ''
        sentence_remainder = []
    
        index_counter = 0

        for token in sent_list:
            if token.dep_ in ['nsubjpass', 'nsubj']:
                recipient_index = index_counter
                recipient = token.lower_
                article_list = [t.text for t in token.lefts]
                try:
                    compound_article = ' '.join(article_list).lower()
                except:
                    compound_article = ''
            if token.dep_ == 'ROOT':
                main_verb_index == index_counter
                main_verb = token.text
                base_form_of_main_verb = token.lemma_
                # TODO: convert VBN to VBD
                # if token.tag_ == 'VBN':
            
            if token.dep_ == 'agent':
                agent_index = index_counter
                agent_children_list = [t.text for t in token.rights]
                agent_children_right = ' '.join(agent_children_list)
                agent = token.text + ' ' + agent_children_right
                agent_children_left = [t.text for t in token.lefts]
                append = True
            if agent and not token.is_punct:
                sentence_remainder.append(token)
            if token.is_punct:
                punct = token.text
            if token.dep_ == 'pobj':
               append = False
               if not agent:
                   agent = token.text
            if token.text in ['may', 'must', 'might', 'could', 'will']:
                inflection = ' ' + token.text + ' '

            index_counter += 1
        if punct in ['?']:
            for token in sent:
                if token.text in ['Can', 'May']:
                    start_word = token.text
                    break

        for key, value in pronouns.items():
            if key in agent.lower() + ' ':
                agent = value.strip()
        index = 0 
        for t in sentence_remainder:
            if t.pos_ in ['DET'] and index >= 2:
                agent_article = t.text
                index += 1

        if recipient_index < agent_index:
            print('-'*50+'\n {}'.format(
                sent)+'\n Active voice skeleton:{} {} {} {}{} {} {}{}'.format(
                    start_word, agent_article, agent, inflection, main_verb, compound_article, recipient, punct).replace(
                        'by ', ''))
            print('Base form of main verb: {}'.format(base_form_of_main_verb))
    except:
        print(sent)
        for token in sent:
            print('Text: ' + token.text + ' |   Dep: ' + token.dep_ + ' |   Tag: ' + token.tag_)

/usr/local/lib/python3.8/dist-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_core_web_sm' (3.4.1) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


As a participant, I do not want the site to shut down while I’m in the midst of a quiz or video so that my progress isn’t interrupted.

Text: As |   Dep: prep |   Tag: IN
Text: a |   Dep: det |   Tag: DT
Text: participant |   Dep: pobj |   Tag: NN
Text: , |   Dep: punct |   Tag: ,
Text: I |   Dep: nsubj |   Tag: PRP
Text: do |   Dep: aux |   Tag: VBP
Text: not |   Dep: neg |   Tag: RB
Text: want |   Dep: ROOT |   Tag: VB
Text: the |   Dep: det |   Tag: DT
Text: site |   Dep: nsubj |   Tag: NN
Text: to |   Dep: aux |   Tag: TO
Text: shut |   Dep: ccomp |   Tag: VB
Text: down |   Dep: prt |   Tag: RP
Text: while |   Dep: mark |   Tag: IN
Text: I |   Dep: nsubj |   Tag: PRP
Text: ’m |   Dep: advcl |   Tag: VBZ
Text: in |   Dep: prep |   Tag: IN
Text: the |   Dep: det |   Tag: DT
Text: midst |   Dep: pobj |   Tag: NN
Text: of |   Dep: prep |   Tag: IN
Text: a |   Dep: det |   Tag: DT
Text: quiz |   Dep: pobj |   Tag: NN
Text: or |   Dep: cc |   Tag: CC
Text: video |   Dep: conj |   Tag: NN

In [128]:
@Language.component("custom_sentencizer")
def custom_sentencizer(doc):
    for i, token in enumerate(doc[:-1]):
    
        if token.text == "."  :
          doc[i + 1].is_sent_start = True
        elif  token.text == "As":
          doc[i].is_sent_start = True
        else:
            # Explicitly set sentence start to False otherwise, to tell
            # the parser to leave those tokens alone
            doc[i + 1].is_sent_start = False
    return doc

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("custom_sentencizer", before="parser")  # Insert before the parser


<function __main__.custom_sentencizer(doc)>

In [129]:
doc = nlp(file)
sentences=[]
for sent in doc.sents:
  sentences.append(sent.text)

In [205]:
class Actor():

  def __init__(self,actorName):
   self.name=actorName
   self.usecases=[]
  def checkSimilarityB2UseCases(self,t1):
    
    t1tokens=nlp(t1)
    print("token -> ",t1) 
    
    for i,x in enumerate(self.usecases):
      v = x.find(t1tokens[0].text)
      if v>-1 :
        self.usecases[i]=x.replace(x,t1)
        print(self.usecases[i])
        return False

    return True

  def addUseCase(self,useCase):
     if self.checkSimilarityB2UseCases(useCase):
        self.usecases.append(useCase)

###############################

In [206]:
actors=[]
for sentenceObj in sentences:
  objNlp=nlp(sentenceObj)
  ### this part get compound nouns (actors))
  for chunk in objNlp.noun_chunks:
    actorObj=Actor(chunk.text)
    if any(obj.name == actorObj.name for obj in actors):
      break
    else:
      actors.append(actorObj)
    break



print("Actors are #################")
print(len(actors))
for act in actors:
  print(act.name)


Actors are #################
9
a participant
an instructor
a site visitor
a search engine
a site owner
a trainer
a buyer
a user
a company


In [207]:
##### nouns 
nouns=[]
for sentenceObj in sentences:
  objNlp=nlp(sentenceObj)
  ### this part get compound nouns (actors))
  for i,chunk in enumerate(objNlp.noun_chunks):
    if i==0:
      continue;
    if chunk.text not in nouns:
      nouns.append(chunk.text)
    else:
      continue

for x in nouns:
  print(x)

I
the site
the midst
a quiz
video
my progress
a forum
a course
questions
topics
the instructor
other participants
announcements
a project “home page
a Scheduled Course
participants
thinks
chapter
an instructor
date
news
quizzes
their learning
part
feedback
material
the previous video(s
it
quiz responses
the database
people
various questions
that
which quizzes
the course outline
a comment
the world
what
each quiz question
the question
an answer
a different order
an FAQ
all my questions
answers
text
a site map
all pages
all feedback
historical or aggregate data
my courses
my account
my current information
a list
all courses
the PDUs
each
me
access
that historical information
someone
videos
those videos
a video course
those quizzes
they
the elements
course
them
an amazing course
a discussion forum
some
not all courses
the schedule
prospective participants
what course
elements
the publication date
course elements
a scheduled course
items
subtitles
any video
the thumbnail image
each video
t

In [208]:
def extractVerb():##### verbs 
  verbs=[]
  for sentenceObj in sentences:
    objNlp=nlp(sentenceObj)
    ### this part get verbs)
    for i,tok in enumerate(objNlp):
      if tok.pos_=="VERB" and tok.lemma_ not in verbs:
        verb=tok.lemma_
        verbs.append(verb)
      else:
        continue

  for x in verbs:
    print(x)

In [209]:
##### compund verbs  phrasalverbs
def extractPhrasalVerbs():
  matcher=Matcher(nlp.vocab)
  compoundVerbs=[]
  for sentenceVar in sentences: 
    v = sentenceVar.find("so that")
    sentenceVar=sentenceVar[0:v]
    x=nlp(sentenceVar)

    pattern0 = [[{"LOWER": "want"}, {"LOWER": "to" },{"POS":"VERB"},{"POS":"DET","OP":"*"},{"POS":"NOUN"}]
                ,[{"LOWER": "make"}, {"POS":"NOUN"},{"POS":"NOUN"}]]

    pattern1=[{"POS":"VERB"},{"POS":"NOUN"}]
    pattern2=[[{"POS":"VERB"},{"POS":"DET"},{"POS":"NOUN"}],[{"POS":"VERB"},{"LOWER":"the"},{"POS":"NOUN"}]]

    pattern3=[{"POS":"NOUN"},{"POS":"NOUN"}]
    matcher.add("verbPhrase", [pattern1])
    matcher.add("verbPhrase2", pattern2)


    matches = matcher(x)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = x[start:end]  # The matched span
        if span.text.find("want the")>-1:
          continue
        compoundVerbs.append(span)
        
  compoundVerbs

In [210]:

def extractVerb(sentence):##### verbs 
  verbs=[]
  objNlp=nlp(sentence)
  ### this part get verbs)
  for i,tok in enumerate(objNlp):
    if tok.pos_=="VERB" and tok.lemma_ not in verbs:
      verb=tok.lemma_
      verbs.append(verb)
    else:
      continue
  return verbs


In [219]:
#usecases of a specific actor

##### compund verbs  phrasalverbs
def extractPhrasalVerbsActor(act):
  matcher=Matcher(nlp.vocab)
  compoundVerbs=[]
  for sentenceVar in sentences: 

    v = sentenceVar.find("so that")
    sentenceVar=sentenceVar[0:v]
    x=nlp(sentenceVar)
    res=sentenceVar.find(act.name)
    if res==-1 :
      continue
    verbs=extractVerb(sentenceVar)
    for v in verbs :
      print(v)
      act.addUseCase(v)

    pattern0 = [{"LOWER": "want"}, {"LOWER": "to" },{"POS":"VERB"},{"POS":"DET","OP":"*"},{"POS":"NOUN"}
                ]
    pattern1=[{"POS":"VERB"},{"POS":"NOUN"}]

    pattern2=[[{"POS":"VERB"},{"POS":"DET"},{"POS":"NOUN"},{"POS":"NOUN","OP":"+"}],[{"POS":"VERB"},{"LOWER":"the"},{"POS":"NOUN"}]]
    pattern3=[{"POS":"NOUN"},{"POS":"NOUN"}]
    pattern4=[[{"LOWER":"want"}, {"LOWER": "the", "OP": "!"}],[{"LOWER":"want"}, {"LOWER": "all", "OP": "!"}]]
    matcher.add("wantto", [pattern0])
    matcher.add("verbPhrase", [pattern1])
    matcher.add("verbPhrase2", pattern2)

    matches = matcher(x)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = x[start:end]  # The matched span
        if span.text.find("want the")>-1 or span.text.find("want all")>-1 :
          continue
        act.addUseCase(span.text)
        break
        


for act in actors:
  extractPhrasalVerbsActor(act)
  print(act.name ,"....")


for act in actors:
  
  print(act.name ,"....")
  print(act.usecases)





want
token ->  want
want
shut
token ->  shut
shut
’m
token ->  ’m
’m
participate
token ->  participate
participate
read
token ->  read
read
token ->  read announcements
read announcements
take
token ->  take
take
see
token ->  see
see
pass
token ->  pass
pass
finish
token ->  finish
finish
take
token ->  take
take
see
token ->  see
see
pass
token ->  pass
pass
take
token ->  take
take
view
token ->  view
view
token ->  viewing the course
viewing the course
leave
token ->  leave
leave
want
token ->  want
want
receive
token ->  receive
receive
token ->  want to receive feedback
want to receive feedback
navigate
token ->  navigate
navigate
submit
token ->  submit
submit
press
token ->  press
press
get
token ->  get
get
token ->  get answers
get answers
manage
token ->  manage
manage
view
token ->  view
view
take
token ->  take
take
earn
token ->  earn
earn
announce
token ->  announce
announce
sign
token ->  sign
sign
re
token ->  re
re
-
token ->  -
-
watch
token ->  watch
watch
want
toke

In [214]:

import re
class UserStory():
  def __init__(self):
   self.actors=[]
   self.usecases=[]
 
  def addActor(self,text):
    if text not in self.actors :
      self.actors.append(text)

  def checkSimilarityB2UseCases(self,t1):
    for x in self.usecases:
      print(t1,"---",x)
      v = x.find(t1)
      if v>-1 :
        return False

    return True

  def addUseCase(self,text):
    if text not in self.usecases and text!="want":
      v = text.find("want the")
      if v >-1:
        return 0
      if self.checkSimilarityB2UseCases(text):
        self.usecases.append(text)
 


   